# Model Definition and Evaluation
## Table of Contents
1. [Model Selection](#model-selection)
2. [Feature Engineering](#feature-engineering)
3. [Hyperparameter Tuning](#hyperparameter-tuning)
4. [Implementation](#implementation)
5. [Evaluation Metrics](#evaluation-metrics)
6. [Comparative Analysis](#comparative-analysis)


In [1]:
!pip install keras-tuner

In [47]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, mean_squared_error, classification_report, mean_absolute_error
import matplotlib.pyplot as plt
# Import models you're considering
from google.colab import drive
drive.mount('/content/drive')
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input
from tensorflow.keras.callbacks import EarlyStopping
import keras_tuner as kt
from sklearn.preprocessing import MinMaxScaler
from kerastuner.tuners import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential


from sklearn.model_selection import GridSearchCV


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Model Selection


We consider a neural network or an LSTM (those were used in the models in the literature review)



## Feature Engineering

Three feature engeneering steps were performed: Instead of using the max und min temperature, the span of temperature of that day was used. --> Tells if the weather was stable or fluctuating that day

The interaction between the average Temperature and the humidity is used -->There is a correlation between these two parameters


The pressure change is also interesting, a decreasing pressure indicates a low pressure area, an increasing pressure indicates a high pressure area


In [38]:
# Load the dataset
# Replace 'your_dataset.csv' with the path to your actual dataset
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Dobersdorf_all_cut.csv', sep=";", decimal=",", header=0)

# Perform any feature engineering steps
# Example: df['new_feature'] = df['feature1'] + df['feature2']
df['temp_range'] = df['Tmax'] - df['Tmin']
df['temp_hum_interaction'] = df['Tavg'] * df['Hum']
##.diff() calculates the difference to the value the day before
df['pressure_change'] = df['Pres'].diff()

# Feature and target variable selection
X = df[['Tavg', 'temp_range','temp_hum_interaction','Sun','Pres','pressure_change','Vis', 'Tmin']]
y = df['Produktion(kWH)']
###values between 0 and 1
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)
y = np.array(y).reshape(-1, 1)
y_scaled = scaler.fit_transform(y)


split_index = int(len(df) * 0.8)
X_train, X_test = X_scaled[:split_index], X_scaled[split_index:]
y_train, y_test = y_scaled[:split_index], y_scaled[split_index:]

# Split the dataset
##Splitting time series data random is problematic
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(len(X_train))
print(len(X_test))
print(len(y_train))
print(len(y_test))
print(X_train)
print(y_train)

###here we had to exclude missing values
X_train = np.nan_to_num(X_train, nan=0.0, posinf=0.0, neginf=0.0)

print("X nan:", np.isnan(X_train).any())
print("y nan:", np.isnan(y_train).any())
print("X inf:", np.isinf(X_train).any())
print("y inf:", np.isinf(y_train).any())

###here we had to exclude missing values
X_test= np.nan_to_num(X_test, nan=0.0, posinf=0.0, neginf=0.0)

print("X nan:", np.isnan(X_train).any())
print("y nan:", np.isnan(y_train).any())
print("X inf:", np.isinf(X_train).any())
print("y inf:", np.isinf(y_train).any())

673
169
673
169
[[0.82040872 0.33918129 0.79795612 ...        nan 0.60631493 0.88501742]
 [0.7607254  0.38011696 0.76776064 ... 0.65543365 0.55495684 0.76655052]
 [0.7711463  0.54385965 0.74285803 ... 0.60309996 0.66921874 0.7456446 ]
 ...
 [0.63594532 0.60818713 0.57047326 ... 0.52995472 0.65903628 0.57142857]
 [0.66274191 0.67836257 0.57353263 ... 0.48719959 0.87209257 0.58536585]
 [0.67668155 0.50292398 0.58493365 ... 0.56739812 0.9432664  0.65853659]]
[[0.6277709 ]
 [0.61471813]
 [0.73905705]
 [0.31799257]
 [0.66310341]
 [0.59120063]
 [0.50602003]
 [0.18780241]
 [0.44829526]
 [0.84741758]
 [0.57297176]
 [0.48992911]
 [0.58017329]
 [0.55800608]
 [0.55946889]
 [0.28198492]
 [0.45617194]
 [0.36975357]
 [0.41172499]
 [0.55924384]
 [0.63733543]
 [0.44210645]
 [0.3148419 ]
 [0.58512434]
 [0.64037358]
 [0.64217396]
 [0.46888714]
 [0.19162822]
 [0.60852931]
 [0.61764375]
 [0.56194441]
 [0.72589175]
 [0.12996512]
 [0.33667154]
 [0.43029144]
 [0.63474738]
 [0.49848093]
 [0.73624395]
 [0.5748

## Hyperparameter Tuning

[Discuss any hyperparameter tuning methods you've applied, such as Grid Search or Random Search, and the rationale behind them.]


In [5]:
!pip install scikeras
!pip install -U scikit-learn scikeras

In [61]:
def dimension_train(X, y, time_step=1):
      X1, y1 = [], []
      X = np.array(X)
      y = np.array(y)  # jetzt 0…n-1
      for i in range(len(X) - time_step):
        X1.append(X[i:i+time_step,:])
        y1.append(y[i+time_step])
      return np.array(X1), np.array(y1)

time_step = 10
X,y=dimension_train(X_train,y_train, time_step)
print(len(X))
print(len(y))

663
663


In [ ]:
# Implement hyperparameter tuning
# Example using GridSearchCV with a DecisionTreeClassifier
# param_grid = {'max_depth': [2, 4, 6, 8]}
# grid_search = GridSearchCV(DecisionTreeClassifier(), param_grid, cv=5)
# grid_search.fit(X_train, y_train)
from scikeras.wrappers import KerasRegressor
from sklearn.model_selection import GridSearchCV
time_step = 10
def model1(units=40,learning_rate=0.1):
  model1 = Sequential()
  model1.add(Input(shape=(time_step, 8)))
  model1.add(LSTM(units, return_sequences=True))
  model1.add(LSTM(units))
  model1.add(Dense(1))
  model1.compile(optimizer = keras.optimizers.Adam( learning_rate=learning_rate), loss='mean_squared_error')
  return model1

model = KerasRegressor(
    model=model1,
    verbose=0
)


###I wanted to test more parameters, but it took so long and the runtime disconnected without finishing###
param_grid = {
    "model__units": [20, 30],
    "batch_size": [8, 16],
    "epochs": [10, 20],
    "model__learning_rate": [0.00001,0.000001]

}

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5)
grid_search.fit(X, y)



In [28]:
print(grid_search.best_params_)

{'batch_size': 8, 'epochs': 20, 'model__learning_rate': 1e-05, 'model__units': 30}


## Implementation

[Implement the final model(s) you've selected based on the above steps.]


In [62]:


# Build the LSTM model(with hyperparameters)
model = Sequential()
model.add(LSTM(30, return_sequences=True, input_shape=(time_step, 8)))
model.add(LSTM(30))
model.add(Dense(1))
model.compile(optimizer= keras.optimizers.Adam(learning_rate=0.00001), loss='mean_squared_error')

# Train the model
model.fit(X, y, epochs=20, batch_size=8)



Epoch 1/20


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


83/83 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - loss: 0.3901
Epoch 2/20
83/83 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 0.3319
Epoch 3/20
83/83 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 0.3034
Epoch 4/20
83/83 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 0.2456
Epoch 5/20
83/83 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - loss: 0.2166
Epoch 6/20
83/83 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.1775
Epoch 7/20
83/83 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.1611
Epoch 8/20
83/83 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - loss: 0.1258
Epoch 9/20
83/83 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.1088
Epoch 10/20
83/83 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0898
Epoch 11/20
83/83 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0805
Epoch 12/20
83/83 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0747
Epoch 13/20
83/83 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 0.0675
Epoch 14/20
83/83 ━━━━━━━━━━━━━━━━━━━━ 2s 21ms/step - loss: 0.0619
Epoch 15/20
83/83 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - loss: 0.0575
Epoch 16/20
83/8

In [63]:
print(len(X_test))
print(len(y_test))
###For the test sets
def dimension_test(X, y, time_step=1):
      X2, y2 = [], []
      X = np.array(X)
      y = np.array(y)
      for i in range(len(X) - time_step):
        X2.append(X[i:i+time_step,:])
        y2.append(y[i + time_step - 1])
      return np.array(X2), np.array(y2)

time_step = 10
X_test1,y_test1=dimension_test(X_test,y_test, time_step)
print(len(X_test1))
print(len(y_test1))


169
169
159
159


## Evaluation Metrics

[Clearly specify which metrics you'll use to evaluate the model performance, and why you've chosen these metrics.]


In [57]:
# Evaluate the model using your chosen metrics
# Example for classification
# y_pred = model.predict(X_test)
# print(classification_report(y_test, y_pred))
y_pred = model.predict(X_test1)
mse = mean_squared_error(y_test1, y_pred)
print("Mean squared error is:",mse)
mae= mean_absolute_error(y_test1, y_pred)
print("Mean abbsolute error is:",mae)

###plotting did not work? (Too many dimensions)



5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
Mean squared error is: 0.04548698887928251
Mean abbsolute error is: 0.17725013478034668


## Comparative Analysis

[Compare the performance of your model(s) against the baseline model. Discuss any improvements or setbacks and the reasons behind them.]


In [ ]:
# Comparative Analysis code (if applicable)
# Example: comparing accuracy of the baseline model and the new model
##Comparing the Mse
##replace with new value
print(f"Baseline Model mse: {baseline_accuracy}, New Model Accuracy: 0.04548")
